In [1]:
!pip install transformers
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 38.8 MB/s 
     |████████████████████████████████| 182 kB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import pipeline

In [3]:
def data_cleaning(val):
  """cleaning the text to remove the irrelevant symbols and characters"""
  val = val.replace(r'[a-zA-Z0-9_]{1,50}',"")
  val = val.replace(r'@([a-zA-Z0-9_]{1,50})',"")
  val = val.replace(r'http\S+', "")
  val = val.replace(r'[:"$%&\*+,-/:;<=>@\\^_`{|}~]+', "")
  return val

#specifically for reddit posts as the text is too big 
def text_reduction(sen):
  """some of the reddit posts are very long and fails the model. This funstion reduces overall text length if is greater than 1200 """
  if len(sen) > 1200:
    return sen[0:1200]
  else:
    return sen

In [4]:
def add_sentiment(df):
  """
  This function labels the text as Strongly negative, negative, neutral, positive, strongly positive based on sentiment score and label 
  determined through the pretrained DistillBERT model.It also updates the sentiment scores of neagtive labels by multiplying it by -1
  to make the score negative. 
  """
  l=[]
  Sentiment_val = ["Strongly Negative", "Negative", "Neutral", "Positive", "Strongly Positive"]
  for index, row in df.iterrows():
     score = row['sentiment_score']
     label = row['sentiment_type']
     if (label == 'POSITIVE'):
        if((score > 0.25) and (score <= 0.75)):
          l.append("Positive")
        elif (score > 0.75):
          l.append("Strongly Positive")
        else:
          l.append("Neutral")
     elif (label == 'NEGATIVE'):
        if((score > 0.25) and (score <= 0.75)):
          l.append("Negative")
        elif (score > 0.75):
          l.append("Strongly Negative")
        else:
          l.append("Neutral") 
     else:
        l.append("Neutral")
  df['sentiment'] = pd.DataFrame(l)
  return df

In [5]:
SentimentClassifier = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [10]:
def FunctionBERTSentiment(inpText):
  """gives the sentiment value of the inpText. output is either negative or positive"""
  return(SentimentClassifier(inpText)[0]['label'])

def FunctionBertSentimentScore(inpText):
   """gives the sentiment score of the inpText. score is between 0 to 1"""
   return(SentimentClassifier(inpText)[0]['score'])
 

In [13]:
###Example on how to run the sentiment analysis model on a twitter tweet csv file. Make sure that the csv file has a column of text
### to run it, you just have to change the csv filename to your current file name 
#prochoice = pd.read_csv("#prochoice.csv") 
#prochoice['text'] = prochoice['text'].apply(data_cleaning)
#prochoice['sentiment_type'] = prochoice['text'].apply(FunctionBERTSentiment)
#prochoice['sentiment_score'] = prochoice['text'].apply(FunctionBertSentimentScore)
#prochoice = add_sentiment(prochoice)     
#prochoice.loc[(prochoice["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = prochoice.sentiment_score * -1
#prochoice = prochoice.drop("sentiment_type", axis=1)
#prochoice.to_csv('output_#prochoice.csv', index = True)

In [ ]:
### script to run it on the sample dataset provided in the zip file. Uncomment the code in this block if you want to run the sentiment analysis on the sample dataset provided 

"""
abortion_choice = pd.read_csv("#Abortion.csv") 
abortion_choice['text'] = abortion_choice['text'].apply(data_cleaning)
abortion_choice['sentiment_type'] = abortion_choice['text'].apply(FunctionBERTSentiment)
abortion_choice['sentiment_score'] = abortion_choice['text'].apply(FunctionBertSentimentScore)
abortion_choice = add_sentiment(abortion_choice)   
abortion_choice.loc[(abortion_choice["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortion_choice.sentiment_score * -1
abortion_choice = abortion_choice.drop("sentiment_type", axis=1)
abortion_choice.to_csv('output_#Abortion.csv', index = True)

abortionHealth_choice = pd.read_csv("#AbortionIsHealthcare.csv") 
abortionHealth_choice['text'] = abortionHealth_choice['text'].apply(data_cleaning)
abortionHealth_choice['sentiment_type'] = abortionHealth_choice['text'].apply(FunctionBERTSentiment)
abortionHealth_choice['sentiment_score'] = abortionHealth_choice['text'].apply(FunctionBertSentimentScore)
abortionHealth_choice = add_sentiment(abortionHealth_choice)   
abortionHealth_choice.loc[(abortionHealth_choice["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionHealth_choice.sentiment_score * -1
abortionHealth_choice = abortionHealth_choice.drop("sentiment_type", axis=1)
abortionHealth_choice.to_csv('output_#AbortionIsHealthcare.csv', index = True)

abortionMurder = pd.read_csv("#AbortionIsMurder.csv") 
abortionMurder['text'] = abortionMurder['text'].apply(data_cleaning)
abortionMurder['sentiment_type'] = abortionMurder['text'].apply(FunctionBERTSentiment)
abortionMurder['sentiment_score'] = abortionMurder['text'].apply(FunctionBertSentimentScore)
abortionMurder = add_sentiment(abortionMurder)   
abortionMurder.loc[(abortionMurder["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionMurder.sentiment_score * -1
abortionMurder = abortionMurder.drop("sentiment_type", axis=1)
abortionMurder.to_csv('output_#AbortionIsMurder.csv', index = True)

abortionHumanRights = pd.read_csv("#AbortionRightsAreHumanRights.csv") 
abortionHumanRights['text'] = abortionHumanRights['text'].apply(data_cleaning)
abortionHumanRights['sentiment_type'] = abortionHumanRights['text'].apply(FunctionBERTSentiment)
abortionHumanRights['sentiment_score'] = abortionHumanRights['text'].apply(FunctionBertSentimentScore)
abortionHumanRights = add_sentiment(abortionHumanRights)   
abortionHumanRights.loc[(abortionHumanRights["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionHumanRights.sentiment_score * -1
abortionHumanRights = abortionHumanRights.drop("sentiment_type", axis=1)
abortionHumanRights.to_csv('output_#AbortionRightsAreHumanRights.csv', index = True)

abortionRights = pd.read_csv("#Abortionrights.csv") 
abortionRights['text'] = abortionRights['text'].apply(data_cleaning)
abortionRights['sentiment_type'] = abortionRights['text'].apply(FunctionBERTSentiment)
abortionRights['sentiment_score'] = abortionRights['text'].apply(FunctionBertSentimentScore)
abortionRights = add_sentiment(abortionRights)   
abortionRights.loc[(abortionRights["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionRights.sentiment_score * -1
abortionRights = abortionRights.drop("sentiment_type", axis=1)
abortionRights.to_csv('output_#Abortionrights.csv', index = True)

abortionBan= pd.read_csv("#Abortionban.csv") 
abortionBan['text'] = abortionBan['text'].apply(data_cleaning)
abortionBan['sentiment_type'] = abortionBan['text'].apply(FunctionBERTSentiment)
abortionBan['sentiment_score'] = abortionBan['text'].apply(FunctionBertSentimentScore)
abortionBan = add_sentiment(abortionBan)   
abortionBan.loc[(abortionBan["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionBan.sentiment_score * -1
abortionBan = abortionBan.drop("sentiment_type", axis=1)
abortionBan.to_csv('output_#Abortionban.csv', index = True)

catholic = pd.read_csv("#CatholicTwitter.csv") 
catholic['text'] = catholic['text'].apply(data_cleaning)
catholic['sentiment_type'] = catholic['text'].apply(FunctionBERTSentiment)
catholic['sentiment_score'] = catholic['text'].apply(FunctionBertSentimentScore)
catholic = add_sentiment(catholic)   
catholic.loc[(catholic["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = catholic.sentiment_score * -1
catholic = catholic.drop("sentiment_type", axis=1)
catholic.to_csv('output_#CatholicTwitter.csv', index = True)

christian = pd.read_csv("#ChristianTwitter.csv") 
christian['text'] = christian['text'].apply(data_cleaning)
christian['sentiment_type'] = christian['text'].apply(FunctionBERTSentiment)
christian['sentiment_score'] = christian['text'].apply(FunctionBertSentimentScore)
christian = add_sentiment(christian)   
christian.loc[(christian["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = christian.sentiment_score * -1
christian = christian.drop("sentiment_type", axis=1)
christian.to_csv('output_#ChristianTwitter.csv', index = True)

abortionEnd = pd.read_csv("#EndAbortion.csv") 
abortionEnd['text'] = abortionEnd['text'].apply(data_cleaning)
abortionEnd['sentiment_type'] = abortionEnd['text'].apply(FunctionBERTSentiment)
abortionEnd['sentiment_score'] = abortionEnd['text'].apply(FunctionBertSentimentScore)
abortionEnd = add_sentiment(abortionEnd)   
abortionEnd.loc[(abortionEnd["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionEnd.sentiment_score * -1
abortionEnd = abortionEnd.drop("sentiment_type", axis=1)
abortionEnd.to_csv('output_#EndAbortion.csv', index = True)

life = pd.read_csv("#Life.csv") 
life['text'] = life['text'].apply(data_cleaning)
life['sentiment_type'] = life['text'].apply(FunctionBERTSentiment)
life['sentiment_score'] = life['text'].apply(FunctionBertSentimentScore)
life = add_sentiment(life)   
life.loc[(life["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = life.sentiment_score * -1
life = life.drop("sentiment_type", axis=1)
life.to_csv('output_#Life.csv', index = True)

proRoe_overturned = pd.read_csv("#RoeOverturned.csv")
proRoe_overturned['text'] = proRoe_overturned['text'].apply(data_cleaning)
proRoe_overturned['sentiment_type'] = proRoe_overturned['text'].apply(FunctionBERTSentiment)
proRoe_overturned['sentiment_score'] = proRoe_overturned['text'].apply(FunctionBertSentimentScore)
proRoe_overturned = add_sentiment(proRoe_overturned)   
proRoe_overturned.loc[(proRoe_overturned["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = proRoe_overturned.sentiment_score * -1
proRoe_overturned = proRoe_overturned.drop("sentiment_type", axis=1)
proRoe_overturned.to_csv('output_#RoeOverturned.csv', index = True)

proRoewade = pd.read_csv("#Roevwade.csv") 
proRoewade['text'] = proRoewade['text'].apply(data_cleaning)
proRoewade['sentiment_type'] = proRoewade['text'].apply(FunctionBERTSentiment)
proRoewade['sentiment_score'] = proRoewade['text'].apply(FunctionBertSentimentScore)
proRoewade = add_sentiment(proRoewade)   
proRoewade.loc[(proRoewade["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = proRoewade.sentiment_score * -1
proRoewade = proRoewade.drop("sentiment_type", axis=1)
proRoewade.to_csv('output_#Roevwade.csv', index = True)

abortionSafe= pd.read_csv("#Safeabortion.csv") 
abortionSafe['text'] = abortionSafe['text'].apply(data_cleaning)
abortionSafe['sentiment_type'] = abortionSafe['text'].apply(FunctionBERTSentiment)
abortionSafe['sentiment_score'] = abortionSafe['text'].apply(FunctionBertSentimentScore)
abortionSafe = add_sentiment(abortionSafe)   
abortionSafe.loc[(abortionSafe["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = abortionSafe.sentiment_score * -1
abortionSafe = abortionSafe.drop("sentiment_type", axis=1)
abortionSafe.to_csv('output_#Safeabortion.csv', index = True)

banabortion = pd.read_csv("#banabortion.csv") 
banabortion['text'] = banabortion['text'].apply(data_cleaning)
banabortion['sentiment_type'] = banabortion['text'].apply(FunctionBERTSentiment)
banabortion['sentiment_score'] = banabortion['text'].apply(FunctionBertSentimentScore)
banabortion = add_sentiment(banabortion)   
banabortion.loc[(banabortion["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = banabortion.sentiment_score * -1
banabortion = banabortion.drop("sentiment_type", axis=1)
banabortion.to_csv('output_#banabortion.csv', index = True)

prolife = pd.read_csv("#prolife.csv", encoding='ISO-8859-1') 
prolife['text'] = prolife['text'].apply(data_cleaning)
prolife['sentiment_type'] = prolife['text'].apply(FunctionBERTSentiment)
prolife['sentiment_score'] = prolife['text'].apply(FunctionBertSentimentScore)
prolife = add_sentiment(prolife)   
prolife.loc[(prolife["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = prolife.sentiment_score * -1
prolife = prolife.drop("sentiment_type", axis=1)
prolife.to_csv('output_#prolife.csv', index = True)

stillLife = pd.read_csv("#Stillife.csv") 
stillLife['text'] = stillLife['text'].apply(data_cleaning)
stillLife['sentiment_type'] = stillLife['text'].apply(FunctionBERTSentiment)
stillLife['sentiment_score'] = stillLife['text'].apply(FunctionBertSentimentScore)
stillLife = add_sentiment(stillLife)   
stillLife.loc[(stillLife["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = stillLife.sentiment_score * -1
stillLife = stillLife.drop("sentiment_type", axis=1)
stillLife.to_csv('output_#Stillife.csv', index = True)

killingBabies = pd.read_csv("#Stopkillingbabies.csv") 
killingBabies['text'] = killingBabies['text'].apply(data_cleaning)
killingBabies['sentiment_type'] = killingBabies['text'].apply(FunctionBERTSentiment)
killingBabies['sentiment_score'] = killingBabies['text'].apply(FunctionBertSentimentScore)
killingBabies = add_sentiment(killingBabies)   
killingBabies.loc[(killingBabies["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = killingBabies.sentiment_score * -1
killingBabies = killingBabies.drop("sentiment_type", axis=1)
killingBabies.to_csv('output_#Stopkillingbabies.csv', index = True)

unbornMatters = pd.read_csv("#UnbornLivesMatter.csv") 
unbornMatters['text'] = unbornMatters['text'].apply(data_cleaning)
unbornMatters['sentiment_type'] = unbornMatters['text'].apply(FunctionBERTSentiment)
unbornMatters['sentiment_score'] = unbornMatters['text'].apply(FunctionBertSentimentScore)
unbornMatters = add_sentiment(unbornMatters)   
unbornMatters.loc[(unbornMatters["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = unbornMatters.sentiment_score * -1
unbornMatters = unbornMatters.drop("sentiment_type", axis=1)
unbornMatters.to_csv('output_#UnbornLivesMatter.csv', index = True)

women = pd.read_csv("#Women.csv") 
women['text'] = women['text'].apply(data_cleaning)
women['sentiment_type'] = women['text'].apply(FunctionBERTSentiment)
women['sentiment_score'] = women['text'].apply(FunctionBertSentimentScore)
women = add_sentiment(women)   
women.loc[(women["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = women.sentiment_score * -1
women = women.drop("sentiment_type", axis=1)
women.to_csv('output_#Women.csv', index = True)

womenReproRights = pd.read_csv("#WomensReproductiveRights.csv") 
womenReproRights['text'] = womenReproRights['text'].apply(data_cleaning)
womenReproRights['sentiment_type'] = womenReproRights['text'].apply(FunctionBERTSentiment)
womenReproRights['sentiment_score'] = womenReproRights['text'].apply(FunctionBertSentimentScore)
womenReproRights = add_sentiment(womenReproRights)   
womenReproRights.loc[(womenReproRights["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = womenReproRights.sentiment_score * -1
womenReproRights = womenReproRights.drop("sentiment_type", axis=1)
womenReproRights.to_csv('output_#WomensReproductiveRights.csv', index = True)

womenRights = pd.read_csv("#Womensrights.csv") 
womenRights['text'] = womenRights['text'].apply(data_cleaning)
womenRights['sentiment_type'] = womenRights['text'].apply(FunctionBERTSentiment)
womenRights['sentiment_score'] = womenRights['text'].apply(FunctionBertSentimentScore)
womenRights = add_sentiment(womenRights)   
womenRights.loc[(womenRights["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = womenRights.sentiment_score * -1
womenRights = womenRights.drop("sentiment_type", axis=1)
womenRights.to_csv('output_#Womensrights.csv', index = True)
"""

In [ ]:
#### Start of the Reddit Data Sentiment Scores ####

In [14]:
###Running the above sentiment model on reddit posts is slightly different. This is the example on how to run the sentiment analysis model on a reddit tweet csv file. 
###Make sure that the csv file has a column of text and title. Also, to run it, you just have to change the csv filename to your current file name 
#reddit_abortion = pd.read_csv("abortion.csv")
#reddit_abortion['body'].fillna("", inplace=True)
#reddit_abortion['body'] = reddit_abortion['title'] + "" + reddit_abortion['body']
#reddit_abortion['body'] = reddit_abortion['body'].apply(data_cleaning)
#reddit_abortion['body'] = reddit_abortion['body'].apply(text_reduction)
#reddit_abortion['sentiment_type'] = reddit_abortion['body'].apply(FunctionBERTSentiment)
#reddit_abortion['sentiment_score'] = reddit_abortion['body'].apply(FunctionBertSentimentScore)
#reddit_abortion = add_sentiment(reddit_abortion)   
#reddit_abortion.loc[(reddit_abortion["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = reddit_abortion.sentiment_score * -1
#reddit_abortion = reddit_abortion.drop("sentiment_type", axis=1)
#reddit_abortion.to_csv('output_abortion.csv', index = True)

In [ ]:
### script to run it on the reddit sample dataset provided in the zip file. Uncomment the code in this block if you want to run the sentiment analysis on the reddit sample dataset provided 

"""
reddit_abortion_debate = pd.read_csv("abortiondebate.csv")
reddit_abortion_debate['body'].fillna("", inplace=True)
reddit_abortion_debate['body'] = reddit_abortion_debate['title'] + "" + reddit_abortion_debate['body']
reddit_abortion_debate['body'] = reddit_abortion_debate['body'].apply(data_cleaning)
reddit_abortion_debate['body'] = reddit_abortion_debate['body'].apply(text_reduction)
reddit_abortion_debate['sentiment_type'] = reddit_abortion_debate['body'].apply(FunctionBERTSentiment)
reddit_abortion_debate['sentiment_score'] = reddit_abortion_debate['body'].apply(FunctionBertSentimentScore)
reddit_abortion_debate = add_sentiment(reddit_abortion_debate)   
reddit_abortion_debate.loc[(reddit_abortion_debate["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = reddit_abortion_debate.sentiment_score * -1
reddit_abortion_debate = reddit_abortion_debate.drop("sentiment_type", axis=1)
reddit_abortion_debate.to_csv('output_abortiondebate.csv', index = True)

reddit_prochoice = pd.read_csv("ProChoice.csv")
reddit_prochoice['body'].fillna("", inplace=True)
reddit_prochoice['body'] = reddit_prochoice['title'] + "" + reddit_prochoice['body']
reddit_prochoice['body'] = reddit_prochoice['body'].apply(data_cleaning)
reddit_prochoice['body'] = reddit_prochoice['body'].apply(text_reduction)
reddit_prochoice['sentiment_type'] = reddit_prochoice['body'].apply(FunctionBERTSentiment)
reddit_prochoice['sentiment_score'] = reddit_prochoice['body'].apply(FunctionBertSentimentScore)
reddit_prochoice = add_sentiment(reddit_prochoice)   
reddit_prochoice.loc[(reddit_prochoice["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = reddit_prochoice.sentiment_score * -1
reddit_prochoice = reddit_prochoice.drop("sentiment_type", axis=1)
reddit_prochoice.to_csv('output_ProChoice.csv', index = True)

reddit_prolife = pd.read_csv("ProLife.csv")
reddit_prolife['body'].fillna("", inplace=True)
reddit_prolife['body'] = reddit_prolife['title'] + "" + reddit_prolife['body']
reddit_prolife['body'] = reddit_prolife['body'].apply(data_cleaning)
reddit_prolife['body'] = reddit_prolife['body'].apply(text_reduction)
reddit_prolife['sentiment_type'] = reddit_prolife['body'].apply(FunctionBERTSentiment)
reddit_prolife['sentiment_score'] = reddit_prolife['body'].apply(FunctionBertSentimentScore)
reddit_prolife = add_sentiment(reddit_prolife)   
reddit_prolife.loc[(reddit_prolife["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = reddit_prolife.sentiment_score * -1
reddit_prolife = reddit_prolife.drop("sentiment_type", axis=1)
reddit_prolife.to_csv('output_ProLife.csv', index = True)

reddit_roewade = pd.read_csv("roevwade2022.csv")
reddit_roewade['body'].fillna("", inplace=True)
reddit_roewade['body'] = reddit_roewade['title'] + "" + reddit_roewade['body']
reddit_roewade['body'] = reddit_roewade['body'].apply(data_cleaning)
reddit_roewade['body'] = reddit_roewade['body'].apply(text_reduction)
reddit_roewade['sentiment_type'] = reddit_roewade['body'].apply(FunctionBERTSentiment)
reddit_roewade['sentiment_score'] = reddit_roewade['body'].apply(FunctionBertSentimentScore)
reddit_roewade = add_sentiment(reddit_roewade)   
reddit_roewade.loc[(reddit_roewade["sentiment_type"] == "NEGATIVE"),"sentiment_score"] = reddit_roewade.sentiment_score * -1
reddit_roewade = reddit_roewade.drop("sentiment_type", axis=1)
reddit_roewade.to_csv('output_roevwade2022.csv', index = True)
"""